In [1]:
import sys
sys.path.append(r"G:\8TH SEMESTER\CityLearn_ML")

In [2]:
from timeseires.utils import t_v_t_split as sp
import numpy as np
import pandas as pd
from pandas import read_csv
import pickle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder
import sys

In [3]:
df = pd.read_csv(r'G:\8TH SEMESTER\CityLearn_ML\Dataset\cleaned_citylearn_data_all_outliers_handled.csv')
print(f"Data shape: {df.shape}")

Data shape: (8760, 26)


In [4]:
df.head()

,non_shiftable_load,outdoor_dry_bulb_temperature_predicted_1,outdoor_dry_bulb_temperature_predicted_2,outdoor_dry_bulb_temperature_predicted_3,outdoor_relative_humidity_predicted_1,outdoor_relative_humidity_predicted_2,outdoor_relative_humidity_predicted_3,diffuse_solar_irradiance_predicted_1,diffuse_solar_irradiance_predicted_2,diffuse_solar_irradiance_predicted_3,...,hour,day_type,electricity_pricing,outdoor_dry_bulb_temperature,solar_generation,carbon_intensity,outdoor_relative_humidity,diffuse_solar_irradiance,direct_solar_irradiance,month
0,2.188750,18.3,22.8,20.0,81.0,68.0,81.0,25.0,683.0,0.0,...,24,7,0.22,20.0,0.0,0.170724,84.0,0.0,0.0,7
1,1.370667,19.4,22.8,19.4,79.0,71.0,87.0,201.0,683.0,0.0,...,1,1,0.22,20.1,0.0,0.157319,79.0,0.0,0.0,8
2,1.101250,21.1,22.2,19.4,73.0,73.0,87.0,420.0,683.0,0.0,...,2,1,0.22,19.7,0.0,0.154503,78.0,0.0,0.0,8
3,0.543217,22.2,22.8,19.4,71.0,71.0,90.0,554.0,522.0,0.0,...,3,1,0.22,19.3,0.0,0.150945,78.0,0.0,0.0,8
4,0.542833,21.7,22.2,18.9,73.0,71.0,90.0,778.0,444.0,0.0,...,4,1,0.22,18.9,0.0,0.151820,78.0,0.0,0.0,8


In [5]:
train_set, validation_set, test_set = sp.t_v_t(df, 70, 20, 10)

In [7]:
train_set.shape


(876, 26)

In [8]:
validation_set.shape

(1752, 26)

In [9]:
test_set.shape

(876, 26)

In [10]:
data_path = r'G:\8TH SEMESTER\CityLearn_ML\Dataset\cleaned_citylearn_data_all_outliers_handled.csv'
df = pd.read_csv(data_path)

In [11]:
# Extract datetime features if 'datetime' exists
if 'datetime' in df.columns:
    df['datetime'] = pd.to_datetime(df['datetime'])
    df['month'] = df['datetime'].dt.month
    df['hour'] = df['datetime'].dt.hour
    df['day_type'] = df['datetime'].dt.dayofweek  # Monday=0, Sunday=6
else:
    print("No 'datetime' column found. Please ensure temporal features exist.")

No 'datetime' column found. Please ensure temporal features exist.


In [12]:
def split_train_val_test(df, train_percent, val_percent, test_percent, random_state=42):
    assert train_percent + val_percent + test_percent == 100, "Percents must sum to 100"
    
    df = df.sample(frac=1, random_state=random_state).reset_index(drop=True)  # shuffle
    n = len(df)
    train_end = int(train_percent / 100 * n)
    val_end = train_end + int(val_percent / 100 * n)
    
    train_set = df.iloc[:train_end]
    val_set = df.iloc[train_end:val_end]
    test_set = df.iloc[val_end:]
    
    return train_set, val_set, test_set

# Split dataset
train_set, validation_set, test_set = split_train_val_test(df, 70, 20, 10)

In [13]:
def cyclical_encode(df, column, max_val):
    df[f'{column}_sin'] = np.sin(2 * np.pi * df[column] / max_val)
    df[f'{column}_cos'] = np.cos(2 * np.pi * df[column] / max_val)
    return df

for dataset in [train_set, validation_set, test_set]:
    for col, max_val in [('month', 12), ('hour', 24), ('day_type', 7)]:
        if col in dataset.columns:
            cyclical_encode(dataset, col, max_val)
    # Drop original columns after encoding
    dataset.drop(columns=['month', 'hour', 'day_type'], errors='ignore', inplace=True)


In [14]:
weather_features = [
    'outdoor_dry_bulb_temperature', 'outdoor_dry_bulb_temperature_predicted_1', 'outdoor_dry_bulb_temperature_predicted_2', 'outdoor_dry_bulb_temperature_predicted_3',
    'outdoor_relative_humidity', 'outdoor_relative_humidity_predicted_1', 'outdoor_relative_humidity_predicted_2', 'outdoor_relative_humidity_predicted_3',
    'diffuse_solar_irradiance', 'diffuse_solar_irradiance_predicted_1', 'diffuse_solar_irradiance_predicted_2', 'diffuse_solar_irradiance_predicted_3',
    'direct_solar_irradiance', 'direct_solar_irradiance_predicted_1', 'direct_solar_irradiance_predicted_2', 'direct_solar_irradiance_predicted_3'
]

econ_env_features = [
    'non_shiftable_load',
    'electricity_pricing', 'electricity_pricing_predicted_1', 'electricity_pricing_predicted_2', 'electricity_pricing_predicted_3',
    'solar_generation',
    'carbon_intensity'
]

# Filter features that exist in train set to avoid errors
weather_features = [f for f in weather_features if f in train_set.columns]
econ_env_features = [f for f in econ_env_features if f in train_set.columns]


In [15]:
# StandardScaler for weather features
weather_scaler = StandardScaler()
train_set[weather_features] = weather_scaler.fit_transform(train_set[weather_features])
validation_set[weather_features] = weather_scaler.transform(validation_set[weather_features])
test_set[weather_features] = weather_scaler.transform(test_set[weather_features])

# MinMaxScaler for economic/environmental features
econ_env_scaler = MinMaxScaler()
train_set[econ_env_features] = econ_env_scaler.fit_transform(train_set[econ_env_features])
validation_set[econ_env_features] = econ_env_scaler.transform(validation_set[econ_env_features])
test_set[econ_env_features] = econ_env_scaler.transform(test_set[econ_env_features])


In [16]:
save_path = r'G:\8TH SEMESTER\CityLearn_ML\Dataset_1'

train_set.to_csv(f'{save_path}\\CityLearn_train.csv', index=False)
validation_set.to_csv(f'{save_path}\\CityLearn_validation.csv', index=False)
test_set.to_csv(f'{save_path}\\CityLearn_test.csv', index=False)

print("Normalized train, validation, and test datasets saved successfully.")


Normalized train, validation, and test datasets saved successfully.


In [20]:
import pickle
import os

In [21]:
input_path = r'G:\8TH SEMESTER\MACHINE_LEARNING(T+L)\LAB_ML\OPENEND_LAB\DATASET\cleaned_non_shiftable_load_v2.csv'
output_dir = r'G:\8TH SEMESTER\CityLearn_ML\Dataset_1'
scaler_path = os.path.join(output_dir, 'scaler.pkl')

# Load dataset
df = pd.read_csv(input_path)

# Split dataset
def split_data(df, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
    total = len(df)
    train_end = int(total * train_ratio)
    val_end = train_end + int(total * val_ratio)
    return df.iloc[:train_end], df.iloc[train_end:val_end], df.iloc[val_end:]

train_df, val_df, test_df = split_data(df)

# Fit scaler on 'non_shiftable_load'
scaler = MinMaxScaler()
scaler.fit(train_df[['non_shiftable_load']])

# Save scaler using pickle
with open(scaler_path, 'wb') as f:
    pickle.dump(scaler, f)

print(f"Scaler saved successfully at: {scaler_path}")

Scaler saved successfully at: G:\8TH SEMESTER\CityLearn_ML\Dataset_1\scaler.pkl
